In [2]:
%pylab

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [3]:
import numpy as np
from scipy import ndimage
from math import ceil

In [25]:
def createColorPattern(color, colorMask):
    maskBase = map(lambda x: x == color, list(colorMask))
    return np.array(maskBase).reshape((2, 2))

def maskFromPattern(colorPattern, imageShape):
    timesW = ceil(float(imageShape[0]) / 2)
    timesH = ceil(float(imageShape[1]) / 2)

    result = np.tile(colorPattern, (timesW, timesH))

    if result.shape[0] == (imageShape[0] + 1):
        result = result[0:-1, :]

    if result.shape[1] == (imageShape[1] + 1):
        result = result[:, 0:-1]

    return result * 1

def demosaicLinearly(rawImage, colorSetup='rggb'):
    rPattern = createColorPattern('r', colorSetup)
    gPattern = createColorPattern('g', colorSetup)
    bPattern = createColorPattern('b', colorSetup)
    
    redMask   = maskFromPattern(rPattern, rawImage.shape)
    greenMask = maskFromPattern(gPattern, rawImage.shape)
    blueMask  = maskFromPattern(bPattern, rawImage.shape)
    
    trueRedPixels   = rawImage * redMask
    trueGreenPixels = rawImage * greenMask
    trueBluePixels  = rawImage * blueMask
        
    # + values are for edge cases (i.e. edge on the left)
    # x values are for ordinary cases.
    redBlueKernel = np.array([
       [0.25, 0.50, 0.25],
       [0.50, 1.00, 0.50],
       [0.25, 0.50, 0.25]
    ], dtype=np.float)
    
    greenKernel = np.array([
       [0.00, 0.25, 0.00],
       [0.25, 1.00, 0.25],
       [0.00, 0.25, 0.00]
    ], dtype=np.float)

    redConvolve   = ndimage.filters.convolve(trueRedPixels, redBlueKernel)
    greenConvolve = ndimage.filters.convolve(trueGreenPixels, greenKernel)
    blueConvolve  = ndimage.filters.convolve(trueBluePixels, redBlueKernel)

    demosaicedImage = np.empty(shape=(rawImage.shape[0], rawImage.shape[1], 3))
    
    demosaicedImage[:, :, 0] = redConvolve
    demosaicedImage[:, :, 1] = greenConvolve
    demosaicedImage[:, :, 2] = blueConvolve

    return demosaicedImage

In [26]:
files = {
    './lighthouse_RAW_noisy_sigma0.01.png': 'rggb',
#    './raw/signs.png': 'grbg',
    './raw/signs-small.png': 'bggr',
#    './raw/text.png': 'gbrg',
#    './raw/text2.png': 'gbgr'
}

for fileName, colorSetup in files.iteritems():
    rawImage = ndimage.imread(fileName)
    print fileName, rawImage.shape
    coloredImage = demosaicLinearly(rawImage, colorSetup)
    imsave(fileName + '.linear.mine.png', coloredImage / 255.0)


./lighthouse_RAW_noisy_sigma0.01.png (768, 512)
./raw/signs-small.png (520, 426)


In [39]:
def applyGamma(image, gamma = 1/2.2):
    red = image[:, :, 0]
    green = image[:, :, 1]
    blue = image[:, :, 2]
    gamma = 1 / 2.2

    red = np.power(red, gamma)
    green = np.power(green, gamma)
    blue = np.power(blue, gamma)

    result = np.empty(shape=(image.shape[0], image.shape[1], 3))
    result[:, :, 0] = red
    result[:, :, 1] = green
    result[:, :, 2] = blue
    
    return result

rawImage = ndimage.imread('./raw/_MG_4257.pgm')
colorSetup = 'rggb'
demosaiced = applyGamma(demosaicLinearly(rawImage, colorSetup) / 255.0)
imshow(demosaiced)
imsave('./raw/_MG_4257.linear.g=2.4.png', demosaiced)